In [1]:
import pickle

import sys
import time
import random

# ---- utility functions 

# given a expression in the form of Reversed Polish Notation "exp_list", return its value
def reversed_polish_notation_evaluation(exp_list):
    
    stack = []
    
    for ele in exp_list:
        
        if ele in ['+', '-', '*', '/']:
            
            if len(stack)>=2:
                post = stack.pop()
                pre = stack.pop()
            else:
                return None
            
            if ele == '+':
                stack.append(pre + post)
                
            elif ele == '-':
                stack.append(pre - post)
            
            elif ele == '*':
                stack.append(pre * post)
            
            elif ele == '/':
                
                if abs(post)>1e-3 and pre%post == 0:
                    stack.append(pre/post)
                else:
                    return None
            
        else:
            stack.append(ele)
    
    if len(stack) == 1:
        return stack[-1]
    else:
        return None
    
# priority comparison of two operators
def opt_compare(opt_a, opt_b):
    
    if (opt_a in ['+', '-'] and opt_b in ['+', '-']) or (opt_a in ['*', '/'] and opt_b in ['*', '/']):
        return 0
    elif opt_a in ['+', '-'] and opt_b in ['*', '/']:
        return -1
    elif opt_a in ['*', '/'] and opt_b in ['+', '-']:
        return +1
    
# translate a RPN expression to the orindary formula expression
def RPN_to_plain_exp(rpn_exp):
    
    stack = []
    stack_opt = []
    
    for ele in rpn_exp:
        
        if ele in ['+', '-', '*', '/']:
            
            if len(stack)>=2:
                post = stack.pop()
                pre = stack.pop()
            else:
                return None
            
            tmp = []
            
            if type(post) == list:
                
                post_opt = stack_opt.pop()
                
                if opt_compare(ele, post_opt) == 1 or (ele == '-' and opt_compare(ele, post_opt) == 0) :
                    tmp = tmp + ['('] + post + [')']
                else:
                    tmp = tmp + post
            else:
                tmp = tmp + [post]
            
            tmp = [ele] + tmp
            
            if type(pre) == list:
                
                if opt_compare(ele, stack_opt.pop()) == 1:
                    tmp = ['('] + pre + [')'] + tmp
                else:
                    tmp = pre + tmp
            else:
                tmp = [pre] + tmp
            
            stack.append(tmp)
            stack_opt.append(ele)
            
        else:
            stack.append(ele)
    
    exp_str = str(stack[-1])
    
    return exp_str[1:-1].replace(", ", "").replace("'", "")

# test the value of a RPN expression
def test_expression_target(exp_set, target):
    
    error_flag = False
    
    for tmp_exp in exp_set:
        
        tmp_val = reversed_polish_notation_evaluation(tmp_exp)
        
        if target != tmp_val:
            error_flag = True
            print 'Error on ', tmp_val, tmp_exp
    
    if not error_flag:
        print 'No incorrect expressions'
    
    
# given a RPN expression list, return if additional operator can be added
def add_operator(exp_list):
    
    cnt_op = 0
    cnt_num = 0
    for i in exp_list:
        if i in ['+', '-', '*', '/']:
            cnt_op += 1
        else:
            cnt_num +=1
            
    return True if cnt_op < cnt_num - 1 else False

# residual of the flag vector    
def residual_flag(flag_list):
    return [1-i for i in flag_list]


# ---- search memthods
            
'''
Depth first search with RPN and memorization

Key techniques:

Reversed Polish Notation (RPN): to handle order of numbers and brackets with ease

Depth first search with memorization: to avoid duplicate search pathes and computation

'''

def exp_search_rpn_memo(list_int, exp, memo, flag, target, bool_memo):
    
    '''
      Arguments:
      
      list_int: list of expressions, each expression is a list object
      exp: expression on the search path
      memo: dictionary for memorizing visited states during the search
      flag: vector (list) of binary values, 1 indicates the selection of certain number
      target: the value to rearch
      bool_memo: use memorization or not
      
      Return:
      
      list of qualified expressions
    '''
        
    exp_res = []
    
    if sum(flag) >= len(list_int) and add_operator(exp) == False:
        return exp_res
    
    add_opt = False
    
#     layer_cnt[0] += 1
#     local_layer = layer_cnt[0]
#     print '+++',local_layer, flag, target
    
    for idx, k in enumerate(list_int + ['+', '-', '*', '/']):
        
        # add operators  
        if k in ['+', '-', '*', '/'] and add_operator(exp) == True:
            
            exp.append(k)
            add_opt = True
                
        # add numbers
        elif k not in ['+', '-', '*', '/'] and sum(flag) < len(list_int) and flag[idx] == 0:
            
            exp.append(k)
            flag[idx] = 1
            add_opt = False
            
        else:
            continue
            
        if bool_memo == True:
            
            resi_flag = residual_flag(flag)
            state = str(resi_flag)
                    
            exp_val = reversed_polish_notation_evaluation(exp)
            
#             print exp, flag
            
            if exp_val != None:
                
                if target == exp_val:
                    
                    if sum(flag) >= len(list_int):
                                                
                        # note: deep copy
                        exp_res.append([j for j in exp])
                    
                    # add operators  
                    if add_opt == True:
                        exp.pop()
                
                    # add numbers
                    else:
                        exp.pop()
                        flag[idx] = 0 
                    
                    continue
                
                
                if state not in memo:
                    memo[state] = {}
                    
                # Depth first search with memorization
                    
                if target * exp_val not in memo[state]:
                    memo[state][target * exp_val] = exp_search_rpn_memo(list_int, [], memo, \
                                                                   flag, target * exp_val, bool_memo)
                for tmp_exp in memo[state][target * exp_val]:
                    exp_res.append(tmp_exp + exp + ['/'])    
                        
                
                if abs(target)>1e-3 and exp_val % target ==0:
                    if exp_val/target not in memo[state]:
                        memo[state][exp_val / target] = exp_search_rpn_memo(list_int, [], memo, \
                                                                   flag, exp_val / target, bool_memo)
                    for tmp_exp in memo[state][exp_val / target]:
                        exp_res.append(exp  + tmp_exp + ['/'])
                        
                if abs(exp_val)>1e-3 and target % exp_val ==0:
                    if target/exp_val not in memo[state]:
                        memo[state][target/exp_val] = exp_search_rpn_memo(list_int, [], memo, \
                                                                   flag, target/exp_val, bool_memo)
                    for tmp_exp in memo[state][target/exp_val]:
                        exp_res.append(exp  + tmp_exp + ['*'])
                        
                
                if exp_val > target:
                    
                    if exp_val - target not in memo[state]:
                        memo[state][exp_val - target] = exp_search_rpn_memo(list_int, [], memo, \
                                                                   flag, exp_val - target, bool_memo)
                    for tmp_exp in memo[state][exp_val - target]:
                        exp_res.append(exp  + tmp_exp + ['-'])
                        
                else:
                    
                    if target - exp_val not in memo[state]:
                        memo[state][target - exp_val] = exp_search_rpn_memo(list_int, [], memo, \
                                                                   flag, target - exp_val, bool_memo)
                    for tmp_exp in memo[state][target - exp_val]:
                        exp_res.append(exp  + tmp_exp + ['+'])

            else:
                exp_res = exp_res + exp_search(list_int, exp, memo, flag, target, bool_memo)
                        
        else:
            exp_search(list_int, exp, memo, flag, target, bool_memo)
            
            
        # reset the expression for the next search  
        if  add_opt == True:
            exp.pop()                
        else:
            exp.pop()
            flag[idx] = 0    
        
#     print '---', local_layer, target, exp_res
    
    return exp_res

'''
Plain brute-force depth first search with RPN
'''
def exp_search_plain(list_int, exp, flag, depth, target, result):
    
    '''
      Arguments:
      
      list_int: list of expressions, each expression is a list object
      exp: expression on the search path
      flag: vector (list) of binary values, 1 indicates the selection of certain number
      depth: counter of chosen numbers
      target: the value to rearch
      result: list for storing qualified expression
      
      Return:
      
      list of qualified expressions
    '''
    
    if depth >= len(list_int) and add_operator(exp) == False:
        
        if reversed_polish_notation_evaluation(exp) == target:
            result.append([i for i in exp])
            
        return
    
    if depth < len(list_int):
        
        for idx_i, i in enumerate(list_int):
            
            if flag[idx_i] == 0:
                
                exp.append(i)
                flag[idx_i] = 1
                
                exp_search_plain(list_int, exp, flag, depth + 1, target, result)
            
                exp.pop()
                flag[idx_i] = 0
     
        
    if add_operator(exp) == True:
        
        for k in ['+', '-', '*', '/']:
            
            exp.append(k)
            
            exp_search_plain(list_int, exp, flag, depth, target, result)
            
            exp.pop()


# the main method for searching                   
def solver(list_int, target, bool_memo):
    
    exp = []
    memo = dict()
    
    flag = [0 for i in range(len(list_int))]
    
    if bool_memo:
        
        results = exp_search_rpn_memo(list_int, exp, memo, flag, target, bool_memo)
        pickle.dump(results, open("exp_rpn.p", "wb"))
        
    else:
        res = []
        
        exp_search_plain(list_int, exp, flag, 0, target, res)
        pickle.dump(res, open("exp_plain.p", "wb"))
        

In [2]:
# ---- set-up 

num_list = [4, 5, 23, 10]
target = 7


In [3]:
# plain

st_time = time.time()

solver(num_list, target, bool_memo = False)

ed_time = time.time()

print "Time elapsed:", ed_time - st_time

# test
exp_plain = pickle.load(open("exp_plain.p", "rb"))
print(len(exp_plain))

test_expression_target(exp_plain, target)


Time elapsed: 0.0683701038361
26
No incorrect expressions


In [4]:
# RPN to ordinary expressions

exp_set = set()

for tmp_exp in exp_plain:
    
    exp_set.add(RPN_to_plain_exp(tmp_exp))
#     print tmp_exp, RPN_to_plain_exp(tmp_exp)

# print len(exp_plain), len(exp_set)
print exp_set


set(['(23-5+10)/4', '(10-5+23)/4', '(10-4)*5-23', '4*5-23+10', '(10-(5-23))/4', '5*(10-4)-23', '5*4-23+10', '5*4+10-23', '4*5-(23-10)', '(10+23-5)/4', '5*4-(23-10)', '4*5+10-23', '10+5*4-23', '(23-(5-10))/4', '10-(23-5*4)', '(23+10-5)/4', '10-23+5*4', '10-23+4*5', '10+4*5-23', '10-(23-4*5)'])


In [5]:
# memorization

st_time = time.time()

solver(num_list, target, True)

ed_time = time.time()

print "Time elapsed:", ed_time - st_time

# test
exp_rpn = pickle.load(open("exp_rpn.p", "rb"))
print(len(exp_rpn))

test_expression_target(exp_rpn, target)


Time elapsed: 0.00829005241394
4
No incorrect expressions


In [6]:
# RPN to ordinary expressions

exp_set = set()

for tmp_exp in exp_rpn:
    
    exp_set.add(RPN_to_plain_exp(tmp_exp))
#     print tmp_exp, RPN_to_plain_exp(tmp_exp)

# print len(exp_plain), len(exp_set)
print exp_set

set(['(23+10-5)/4', '(10+23-5)/4', '10-(23-4*5)', '10-(23-5*4)'])


In [ ]:
# ---- scalability

max_val = 200

for tmp_cnt in range(3, 10):
    
    print "\n---- amount of numbers :", tmp_cnt
        
    num_list = [int(random.random()*max_val) for _ in range(tmp_cnt)]
    target = int(random.random()*max_val)
    
    # plain
    
    st_time = time.time()
    solver(num_list, target, bool_memo = False)
    ed_time = time.time()

    print "Time elapsed of brute-force search:", ed_time - st_time

    # test
    exp_plain = pickle.load(open("exp_plain.p", "rb"))
    test_expression_target(exp_plain, target)
    
    # memorization
    
    st_time = time.time()
    solver(num_list, target, True)
    ed_time = time.time()

    print "Time elapsed of memorization search:", ed_time - st_time

    # test
    exp_rpn = pickle.load(open("exp_rpn.p", "rb"))
    test_expression_target(exp_rpn, target)


---- amount of numbers : 3
Time elapsed of brute-force search: 0.00210881233215
No incorrect expressions
Time elapsed of memorization search: 0.00128388404846
No incorrect expressions
---- amount of numbers : 4
Time elapsed of brute-force search: 0.0759971141815
No incorrect expressions
Time elapsed of memorization search: 0.00243616104126
No incorrect expressions
---- amount of numbers : 5
Time elapsed of brute-force search: 2.20262408257
No incorrect expressions
Time elapsed of memorization search: 0.0153779983521
No incorrect expressions
---- amount of numbers : 6
Time elapsed of brute-force search: 202.105295181
No incorrect expressions
Time elapsed of memorization search: 0.183399915695
No incorrect expressions
---- amount of numbers : 7
Time elapsed of brute-force search: 16600.5010798
No incorrect expressions
Time elapsed of memorization search: 1.56684398651
No incorrect expressions
---- amount of numbers : 8


In [ ]:
# draft code for back-up

def state_exp(exp_list):
    
    stack = []
    
    for ele in exp_list:
        
        if ele in ['+', '-', '*', '/']:
            
            if len(stack)>=2:
                post = stack.pop()
                pre = stack.pop()
            else:
                return None
            
            if ele == '+':
                stack.append(pre + post)
                
            elif ele == '-':
                stack.append(pre - post)
            
            elif ele == '*':
                stack.append(pre * post)
            
            elif ele == '/':
                stack.append(int(1.0*pre / (post+1e-5)))
            
        else:
            stack.append(ele)    
    
    return stack 


def residual_state_exp(list_int, state):
    
    resi_list = []
    
    for i in list_int:
        if i not in state:
            resi_list.append(i)
    
    return sorted(resi_list)

def valid_rpn(exp_list):
    
    cnt_op = 0
    cnt_num = 0
    num_list = []
    
    for i in exp_list:
        if i in ['+', '-', '*', '/']:
            cnt_op += 1
        else:
            cnt_num +=1
            num_list.append(i)
            
    return True if cnt_op == cnt_num-1 else False, sorted(num_list) 

def exp_search(list_int, exp, memo, flag, depth, target):
    
    if depth == 0:
                
        for idx_i, i in enumerate(list_int):
            for idx_j, j in enumerate(list_int):
                
                if i != j:
                    
                    exp.append(i)
                    exp.append(j)
                    
                    flag[idx_i] = 1
                    flag[idx_j] = 1
                    
                    state = sorted([i, j])
                    
                    print i,j 
                    
                    if str(state) not in memo:
                        memo[str(state)] = {}
                        memo[str(state)][0] = []
                    
                    for k in ['+', '-', '*', '/']:
                        
                        exp.append(k)
                        
                        tmpval = reversed_polish_notation_evaluation(exp)
                        
                        if tmpval in memo[str(state)]:
                            memo[str(state)][tmpval].append(exp)
                        else:
                            memo[str(state)][tmpval] = [exp]
                            
                        exp_search(list_int, exp, memo, flag, depth+1, target)
                        exp.pop()
                    
                    exp.pop()
                    exp.pop()
                    
                    flag[idx_i] = 0
                    flag[idx_j] = 0
                        
    elif depth >= len(list_int)-1:
        
        if reversed_polish_notation_evaluation(exp) == target:
            print 'success', exp
            
        return
    
    else:
        
        for idx_i, i in enumerate(list_int):
            
            if flag[idx_i] == 0:
                
                exp.append(i)
                flag[idx_i] = 1
                
                state = state_exp(exp)
                
                if str(state) not in memo:
                    memo[str(state)] = {}
                    memo[str(state)][0] = []
                
                
                resi_state = residual_state_exp(list_int, state)
                
                memo[str(state)] = {}
                memo[str(state)][0] = []
                
                for k in ['+', '-', '*', '/']:
                    
                    exp.append(k)
                    tmpval = reversed_polish_notation_evaluation(exp) 
                    
                    if tmpval in memo[str(state)]:
                        memo[str(state)][tmpval].append(exp)
                    else:
                        memo[str(state)][tmpval] = [exp]
                    
#                     if str(resi_state) in memo and target - tmpval not in memo[str(resi_state)]:
                        
#                         exp_construct()
#                         print "success!", exp
#                         exp.pop()
#                         continue
                    
                    exp_search(list_int, exp, memo, flag, depth + 1, target)
                    exp.pop()
                    
                exp.pop()
                flag[idx_i] = 0

def exp_search_plain(list_int, exp, memo, flag, depth, target, bool_memo, result):
    
    if depth >= len(list_int) and add_operator(exp) == False:
        
        if reversed_polish_notation_evaluation(exp) == target:
            result.append(exp)
            
        return
    
    if depth < len(list_int):
        
        for idx_i, i in enumerate(list_int):
            
            if flag[idx_i] == 0:
                
                exp.append(i)
                flag[idx_i] = 1
                
                
                if bool_memo == True:
                    
                    state = str(flag)
                    
                    if state not in memo:
                        
                        # memorization
                        memo.add(state)
                        
                        # depth first search
                        exp_search_plain(list_int, exp, memo, flag, depth + 1, target, bool_memo, result)
                
                else:
                    exp_search_plain(list_int, exp, memo, flag, depth + 1, target, bool_memo, result)
            
                exp.pop()
                flag[idx_i] = 0
     
        
    if add_operator(exp) == True:
        
        for k in ['+', '-', '*', '/']:
            
            exp.append(k)
            
            # state = str(flag) + str(state_exp(exp))
            
            if bool_memo == True:
                
                state = str(flag) + str(state_exp(exp))
                
                
                if flag[0] == 1 and flag[1] == 1 and flag[2] == 1:
                        print state
                          
                if state not in memo:
                    
                    # memorization
                    memo.add(state)
                    
#                     print memo
                    
                    # depth first search
                    exp_search_plain(list_int, exp, memo, flag, depth, target, bool_memo, result)
            else:
                exp_search_plain(list_int, exp, memo, flag, depth, target, bool_memo, result)
                
            
            exp.pop()